# Cone beam with 2 detectors: longitudinal + transversal

-----------------------------------------------------------------------------------------------------------------------

<u>*The material provided in this notebook can be freely used and modified for educational purposes only. Please cite any content of the notebook as follows:*</u>

- *Panetta D, Camarlinghi N. 3D Image Reconstruction for CT and PET : A Practical Guide with Python. CRC Press; 2020. Available from: https://www.taylorfrancis.com/books/9780429270239*

*For questions, notifications of bugs, or even just for feedback, please contact the authors directly (daniele.panetta@ifc.cnr.it; niccolo.camarlinghi@gmail.com)*

-----------------------------------------------------------------------------------------------------------------------

In [ ]:
import sys
import numpy as np
import os
sys.path.append("../") # this to be able to include all the object contained in the modules
from Misc.Utils import Unpickle,ReadImage,DownscaleImage, OutputFileName
from Misc.Preview import Visualize3dImage
import matplotlib.pyplot as plt
# set the default size of all the plots 5x5 inches
plt.rcParams['figure.figsize'] = [5, 5]
from Algorithms.SinogramGenerator_3D_long_transv import SinogramGenerator_3D
from Algorithms.FBP import FBP
from Geometry.ExperimentalSetupCT_3D_long_transv import ExperimentalSetupCT_3D, Mode, DetectorShape
from Algorithms.MLEM import MLEM
from Misc.DataTypes import voxel_dtype


### Create a CT experimental setup

In this case the value of the ```mode``` member of ```my_experimental_setup``` must be set to ```Mode.CONEBEAM```. The chosen detector shape for this example is ```DetectorShape.PLANAR```.
SDD and SAD denote the source-to-detector and source-to-axis distances, respectively. The ```fan_angle_deg```represents the angle of emission of photons. 

The actual size of the detector row is a derived parameter, calculated on top of ```pixels_per_slice_nb``` and ```fan_angle_deg``` as ```2*sdd*tan(fan_angle/2)```. 

The relationship between number of voxels, voxel size and volume size is the same as in the previous examples.

In [ ]:
# create CT experimental setup
my_experimental_setup = ExperimentalSetupCT_3D()
my_experimental_setup.mode = Mode.CONEBEAM
my_experimental_setup._detector_number = 2
# detector 
my_experimental_setup.pixels_per_slice_nb=100
my_experimental_setup.detector_slice_nb=100
my_experimental_setup.slice_pitch_mm=3
my_experimental_setup.detector_shape=DetectorShape.PLANAR
# number of rotation of the gantry
my_experimental_setup.gantry_angles_nb = 8
# range of the rotation
my_experimental_setup.angular_range_deg = 360
# fov size in mm 
my_experimental_setup.image_matrix_size_mm = np.array([100,100,100])
# voxel size in mm
my_experimental_setup.voxel_size_mm = np.array([1,1,1])
# sources 
my_experimental_setup.sdd_mm=
my_experimental_setup.sad_mm=100
my_experimental_setup.fan_angle_deg=90
# compute the geometry
my_experimental_setup.Update()
print(my_experimental_setup.GetInfo())

You may have noticed that in this example, a voxel size of 4 mm has been chosen along each axis. Indeed, in order to keep the forward and back projection within reasonable time in this single-core, non parallelised educational implementation, we will use a rather coarse reconstruction grid.

### Display the experimental setup

As already done in the previous demos, let's look at the geometry using the ```Draw``` method:

In [ ]:
my_experimental_setup.Draw(use_jupyter=0,camera_pos_mm=(0,-800,400))

### Load the image used to generate cone beam projection data

Let us now load another version of the voxelised 3D phantom. This version is stored in a grid of 100$^3$ voxels (let's assume that each voxel has a side of 1 mm in each direction).

In [ ]:
input_file_name = '../Data/cilinder.npz'
input_img = np.load(input_file_name)['matrix']
input_img = np.transpose(input_img, (2, 1, 0))

In [ ]:
#input_img = np.load('/home/eleonora/eFLASH_3D_Sim-build/Dose_Map_30mm/spettro_9MeV/doseDistribution.npz')['doseDistr'].astype(np.float64)

fig, ax = plt.subplots(1, 3, figsize=(6,12))
ax[0].imshow(input_img[50,:,:])
ax[1].imshow(input_img[:,50,:])
ax[2].imshow(input_img[:,:,50])

print(input_img.shape)

In [ ]:
input_img = np.fromfile("../Data/SheppLogan3D_100x100x100_16bit_us.raw",dtype=np.uint16).reshape ((100,100,100))
input_img=input_img.astype(voxel_dtype)

fig, ax = plt.subplots(1, 3, figsize=(6,12))
input_img = np.transpose(input_img, (2, 1, 0))
#rotated_img_90 = np.transpose(input_img, (0, 2, 1))[:, :, ::-1
ax[0].imshow(input_img[50,:,:])
#ax[0].invert_yaxis() 
ax[1].imshow(input_img[:,50,:])
#ax[1].invert_yaxis() 
ax[2].imshow(input_img[:,:,50])
#ax[2].invert_yaxis() 
print(input_img.shape)

### Generate and display the projection data
Let us now clarify the meaning of the term *sinogram* in the current geometry. We are actually generating a set of projections of the 3D object, giving rise to a 3D array of line integrals. The shape of the generated array follows the ordering: ```(n_of_radial_bins, n_of gantry_angles, n_of_bins_along_axial_direction)```. That is, the first two dimensions are those used in 2D sinograms as already seen in the previous example. But in cone-beam geometry, we often refer to projection data as *radiographs* of the 3D object, and hence we would like to visualise those data using the ordering ```(n_of_radial_bins, n_of_bins_along_axial_direction, n_of gantry_angles)```. This can be easily done with the ```Draw``` function by just putting ```slice_axis=1``` as the second argument (unlike the previous examples where ```slice_axis=2``` was used).

In [ ]:
s=SinogramGenerator_3D(my_experimental_setup)
sino_list=s.GenerateObjectSinogram(input_img,transponse_image=0)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12,12))
ax[0].imshow(sino_list[0]._data[:,0,:])
ax[1].imshow(sino_list[1]._data[:,0,:])
ax[2].imshow(sino_list[1]._data[:,1,:])

fig, ax = plt.subplots(1, 3, figsize=(6,12))
ax[0].imshow(sino_list[1]._data[:,2,:])
ax[1].imshow(sino_list[1]._data[:,3,:])
ax[2].imshow(sino_list[1]._data[:,4,:])

fig, ax = plt.subplots(1, 3, figsize=(6,12))
ax[0].imshow(sino_list[1]._data[:,5,:])
ax[1].imshow(sino_list[1]._data[:,6,:])
ax[2].imshow(sino_list[1]._data[:,7,:])



In [ ]:
projections_t = np.concatenate(np.concatenate(np.transpose(sino_list[0]._data, axes=(2,1,0)), axis=1))
projections_l = np.concatenate(np.concatenate(np.transpose(sino_list[1]._data, axes=(2,1,0)), axis=1))

projections = np.concatenate((projections_t, projections_l))

print('projections_t.shape, projectionsl.shape, projections.shape:, ', projections_t.shape, projections_l.shape, projections.shape)

In [ ]:
algorithm="MLEM"
niter=50
initial_value=1

print(my_experimental_setup.gantry_angles_nb)

#Info for saving
d1 = my_experimental_setup.sad_mm - my_experimental_setup.image_matrix_size_mm[0] * 0.5

output_file_name = OutputFileName('../Reconstruction/', input_file_name, d1, niter, my_experimental_setup.gantry_angles_nb, my_experimental_setup.fan_angle_deg)
print(output_file_name)

# Apre il file in modalità scrittura
with open(output_file_name+'.txt', 'w') as f:
    # Reindirizza l'output di print al file
    print(my_experimental_setup.GetInfo(), file=f)

In [ ]:
it = eval( algorithm+ "()")
it.SetExperimentalSetup(my_experimental_setup)
it.SetNumberOfIterations(niter)
it.SetProjectionData(projections)
initial_guess=np.full(it.GetNumberOfVoxels(),initial_value, dtype=voxel_dtype) 
it.SetImageGuess(initial_guess)
it.SetOutputBaseName(output_file_name) # uncomment this line to save images to disk
output_img = it.Reconstruct()

In [ ]:
print(output_img.shape)
index_x = 50
index_y = 50
index_z = 50

vmin, vmax = 0.,1.
fig, ax = plt.subplots(1, 3, figsize=(12,12))
im=ax[0].imshow(output_img[index_x,:,:])#, vmin=vmin, vmax=vmax)
im=ax[1].imshow(output_img[:,index_y,:] )#, vmin=vmin, vmax=vmax)
im=ax[2].imshow(output_img[:,:,index_z])#, vmin=vmin, vmax=vmax)
fig.colorbar(im, ax=[ax[0], ax[1], ax[2]], orientation = 'horizontal')

fig, ax = plt.subplots(1, 3, figsize=(12,12))
im=ax[0].imshow(input_img[index_x,:,:])#, vmin=vmin, vmax=vmax)
im=ax[1].imshow(input_img[:,index_y,:] )#, vmin=vmin, vmax=vmax)
im=ax[2].imshow(input_img[:,:,index_z])#, vmin=vmin, vmax=vmax)
fig.colorbar(im, ax=[ax[0], ax[1], ax[2]], orientation = 'horizontal')

output_img = output_img/output_img.max()
input_img = input_img/input_img.max()

vmin, vmax= -0.5, 0.5
fig, ax = plt.subplots(1, 3, figsize=(12,12))
ax[0].imshow(output_img[index_x,:,:]-input_img[index_x, :, : ], vmin=vmin, vmax=vmax, cmap='RdBu')
ax[1].imshow(output_img[:,index_y,:] - input_img[:, index_y,:], vmin=vmin, vmax=vmax, cmap='RdBu')
im = ax[2].imshow(output_img[:,:,index_z] - input_img[:,:,index_z], vmin=vmin, vmax=vmax, cmap='RdBu')
#fig.subplots_adjust(right=0.8)
#cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(im, ax=[ax[0], ax[1], ax[2]], orientation = 'horizontal')